In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.basics import *

In [3]:
from pathlib import Path

In [4]:
import pandas as pd

# Rossmann

## Data preparation / Feature engineering

Set `PATH` to the path `~/data/rossmann/`. Create a list of table names, with one entry for each CSV that you'll be loading: 
- train
- store
- store_states
- state_names
- googletrend
- weather
- test

For each csv, read it in using pandas (with `low_memory=False`), and assign it to a variable corresponding with its name. Print out the lengths of the `train` and `test` tables.

In [5]:
PATH = Path("~/.fastai/data/rossmann/")

In [6]:
csvs = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']

In [7]:
tables = [pd.read_csv(f"{PATH}/{csv}.csv", low_memory=False) for csv in csvs]

In [8]:
train, store, store_states, state_names, googletrend, weather, test = tables

Turn the `StateHoliday` column into a boolean indicating whether or not the day was a holiday.

In [9]:
train['StateHoliday'] = train['StateHoliday'] != '0'
test['StateHoliday'] = test['StateHoliday'] != '0'
train['SchoolHoliday'] = train['SchoolHoliday'] != 0
test['SchoolHoliday'] = test['SchoolHoliday'] != 0

In [10]:
train['StateHoliday'].value_counts()

False    986159
True      31050
Name: StateHoliday, dtype: int64

In [11]:
train['SchoolHoliday'].value_counts()

False    835488
True     181721
Name: SchoolHoliday, dtype: int64

Print out the head of the dataframe.

In [12]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,False,True
1,2,5,2015-07-31,6064,625,1,1,False,True
2,3,5,2015-07-31,8314,821,1,1,False,True
3,4,5,2015-07-31,13995,1498,1,1,False,True
4,5,5,2015-07-31,4822,559,1,1,False,True


Create a function `join_df` that joins two dataframes together. It should take the following arguments:
- left (the df on the lft)
- right (the df on the right)
- left_on (the left table join key)
- right_on (the right table join key, defaulting to None; if nothing passed, default to the same as the left join key)
- suffix (default to '_y'; a suffix to give to duplicate columns)

In [13]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    return left.merge(right, left_on=left_on, right_on=right_on if right_on is not None else left_on, 
                      how='left', suffixes=('', suffix))

In [14]:
import pandas as pd
df1 = pd.DataFrame({'a': [1,2], 'b': [100.0, 1000.0]})
df2 = pd.DataFrame({'a': [1,1,1,3,3,3], 'b': [10.0, 10.0, 10.0, 20.0, 20.0, 20.0]})
join_df(df1, df2, 'a')

,a,b,b_y
0,1,100.0,10.0
1,1,100.0,10.0
2,1,100.0,10.0
3,2,1000.0,NaN


Join the weather and state names tables together, and reassign them to the variable `weather`.

In [15]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


In [16]:
state_names

,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB
5,Hamburg,HH
6,Hessen,HE
7,MecklenburgVorpommern,MV
8,Niedersachsen,"HB,NI"
9,NordrheinWestfalen,NW


In [17]:
weather = join_df(weather, state_names, left_on='file', right_on='StateName')

Show the first few rows of the weather df.

In [18]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,10.0,24,16,NaN,0.00,6.0,Rain,225,NordrheinWestfalen,NW
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,2.0,26,21,NaN,1.02,7.0,Rain,240,NordrheinWestfalen,NW
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,2.0,23,14,NaN,0.25,7.0,Rain,263,NordrheinWestfalen,NW
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,3.0,16,10,NaN,0.00,7.0,Rain,268,NordrheinWestfalen,NW


In the `googletrend` table, set the `Date` variable to the first date in the hyphen-separated date string in the `week` field. Set the `State` field to the third element in the underscore-separated string from the `file` field. In all rows where `State == NI`, make it instead equal `HB,NI` which is how it's referred to throughout the reset of the data.

In [19]:
googletrend['Date'] = googletrend['week'].str.split(' - ', expand=True)[0]

In [20]:
googletrend['State'] = googletrend['file'].str.split('_', expand=True)[2]

In [21]:
googletrend['State'][googletrend['State'] == 'NI'] = 'HB,NI'

/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
googletrend.head()

,file,week,trend,Date,State
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN


Write a function `add_datepart` that takes a date field and adds a bunch of numeric columns containing information about the date. It should take the following arguments:
- df (the dataframe you'll be modifying)
- fldname (the date field you'll be splitting into new columns)
- drop (whether or not to drop the old date field; defaults to True)
- time (whether or not to add time fields -- Hour, Minute, Second; defaults to False)

It should append ```
['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear','Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']```

Remember the edge cases around the dtype of the field. Specifically, if it's of type DatetimeTZDtype, cast it instead to np.datetime64. If it's not a subtype of datetime64 already, infer it (see `pd.to_datetime`).

In [23]:
type(np.datetime64)

type

In [24]:
Series(np.array([1,2,3], dtype='int64')).dtype

dtype('int64')

In [25]:
ex_date = pd.to_datetime('2019-01-01')

In [26]:
ex_date

Timestamp('2019-01-01 00:00:00')

In [27]:
ex_date

Timestamp('2019-01-01 00:00:00')

In [28]:
def add_datepart(df, fldname, drop=True, time=False):
    fld = df[fldname]
    
    if fld.dtype == pd.DatetimeTZDtype:
        fld = fld.astype(np.datetime64)
        
    if not np.issubdtype(fld.dtype, np.datetime64):
        fld = pd.to_datetime(fld)
        
    fields = [
        'Year', 'Month', 'Week', 
        'Day', 'Dayofweek', 'Dayofyear',
        'Is_month_end', 'Is_month_start', 'Is_quarter_end', 
        'Is_quarter_start', 'Is_year_end', 'Is_year_start'
    ]
    
    for i in fields:
        df[i] = getattr(fld.dt, i.lower())
        
    if drop:
        df.drop(fldname, inplace=True)
    
    return df

In [29]:
googletrend = add_datepart(googletrend, 'Date', False)

Use `add_datepart` to add date fields to the weather, googletrend, train and test tables.

In [30]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,10.0,24,16,NaN,0.00,6.0,Rain,225,NordrheinWestfalen,NW
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,2.0,26,21,NaN,1.02,7.0,Rain,240,NordrheinWestfalen,NW
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,2.0,23,14,NaN,0.25,7.0,Rain,263,NordrheinWestfalen,NW
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,3.0,16,10,NaN,0.00,7.0,Rain,268,NordrheinWestfalen,NW


In [31]:
weather = add_datepart(weather, 'Date', False)

In [32]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,False,True
1,2,5,2015-07-31,6064,625,1,1,False,True
2,3,5,2015-07-31,8314,821,1,1,False,True
3,4,5,2015-07-31,13995,1498,1,1,False,True
4,5,5,2015-07-31,4822,559,1,1,False,True


In [33]:
train = add_datepart(train, 'Date', False)

In [34]:
test = add_datepart(test, 'Date', False)

Print out the head of the weather table.

In [35]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,1,1,1,1,False,True,False,True,False,True
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,1,2,2,2,False,False,False,False,False,False
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,1,3,3,3,False,False,False,False,False,False
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,1,4,4,4,False,False,False,False,False,False
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,1,5,5,5,False,False,False,False,False,False


In the `googletrend` table, the `file` column has an entry `Rossmann_DE` that represents the whole of germany; we'll want to break that out into its own separate table, since we'll need to join it on `Date` alone rather than both `Date` and `Store`.

In [36]:
googletrend

,file,week,trend,Date,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN,2012,12,48,2,6,337,False,False,False,False,False,False
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN,2012,12,49,9,6,344,False,False,False,False,False,False
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN,2012,12,50,16,6,351,False,False,False,False,False,False
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN,2012,12,51,23,6,358,False,False,False,False,False,False
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN,2012,12,52,30,6,365,False,False,False,False,False,False
5,Rossmann_DE_SN,2013-01-06 - 2013-01-12,71,2013-01-06,SN,2013,1,1,6,6,6,False,False,False,False,False,False
6,Rossmann_DE_SN,2013-01-13 - 2013-01-19,53,2013-01-13,SN,2013,1,2,13,6,13,False,False,False,False,False,False
7,Rossmann_DE_SN,2013-01-20 - 2013-01-26,58,2013-01-20,SN,2013,1,3,20,6,20,False,False,False,False,False,False
8,Rossmann_DE_SN,2013-01-27 - 2013-02-02,55,2013-01-27,SN,2013,1,4,27,6,27,False,False,False,False,False,False
9,Rossmann_DE_SN,2013-02-03 - 2013-02-09,55,2013-02-03,SN,2013,2,5,3,6,34,False,False,False,False,False,False


In [37]:
rossmann_full = googletrend[googletrend['file'] == 'Rossmann_DE']

Now let's do a bunch of joins to build our entire dataset! Remember after each one to check if the right-side data is null. This is the benefit of left-joining; it's easy to debug by checking for null rows. Let's start by joining `store` and `store_states` in a new table called `store`.

In [38]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [39]:
store_states.head()

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


In [40]:
store = join_df(store, store_states, 'Store')

In [41]:
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",TH
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NW
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,BE
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,SN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN,SN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN,SH
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN,SH
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN,NW
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN,"HB,NI"


In [42]:
store['State'].isna().sum()

0

Next let's join `train` and `store` in a table called `joined`. Do the same for `test` and `store` in a table called `joined_test`.

In [43]:
joined = join_df(train, store, 'Store')

In [44]:
joined.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,1,5,2015-07-31,5263,555,1,1,False,True,2015,...,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
1,2,5,2015-07-31,6064,625,1,1,False,True,2015,...,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",TH
2,3,5,2015-07-31,8314,821,1,1,False,True,2015,...,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NW
3,4,5,2015-07-31,13995,1498,1,1,False,True,2015,...,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,BE
4,5,5,2015-07-31,4822,559,1,1,False,True,2015,...,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,SN


In [45]:
joined_test = join_df(test, store, 'Store')

In [46]:
joined_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,1,1,4,2015-09-17,1.0,1,False,False,2015,9,...,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
1,2,3,4,2015-09-17,1.0,1,False,False,2015,9,...,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NW
2,3,7,4,2015-09-17,1.0,1,False,False,2015,9,...,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN,SH
3,4,8,4,2015-09-17,1.0,1,False,False,2015,9,...,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN,SH
4,5,9,4,2015-09-17,1.0,1,False,False,2015,9,...,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN,NW


Next join `joined` and `googletrend` on the columns `["State", "Year", "Week"]`. Again, do the same for the test data.

In [47]:
joined = join_df(joined, googletrend, ["State", "Year", "Week"], suffix='_g')
joined_test = join_df(joined_test, googletrend, ["State", "Year", "Week"], suffix='_g')

Join `joined` and `trend_de` on `["Year", "Week"]` with suffix `_DE`. Same for test.

In [48]:
joined = join_df(joined, rossmann_full, 'Date', suffix='_rf')

In [49]:
joined_test = join_df(joined_test, rossmann_full, 'Date', suffix='_rf')

Join `joined` and `weather` on `["State", "Date"]`. Same for test.

In [52]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,1,1,1,1,False,True,False,True,False,True
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,1,2,2,2,False,False,False,False,False,False
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,1,3,3,3,False,False,False,False,False,False
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,1,4,4,4,False,False,False,False,False,False
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,1,5,5,5,False,False,False,False,False,False


In [50]:
joined = join_df(joined, weather, ["State", "Date"])
joined_test = join_df(joined_test, weather, ["State", "Date"])

In [55]:
joined.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
       'week', 'trend', 'Date_g', 'Month_g', 'Day_g', 'Dayofweek_g',
       'Dayofyear_g', 'Is_month_end_g', 'Is_month_start_g', 'Is_quarter_end_g',
       'Is_quarter_start_g', 'Is_year_end_g', 'Is_year_start_g', 'file_rf',
       'week_rf', 'trend_rf', 'State_rf', 'Year_rf', 'Month_rf', 'Week_rf',
       'Day_rf', 'Dayofweek_rf', 'Dayofyear_rf', 'Is_month_end_rf',
       'Is_month_start_rf', 'Is_quarter_end_rf', 'Is_quarter_start_rf',
       'Is_year_end_rf', 'Is_year_start_rf', 'file_y', 'M

In [56]:
joined['Min_DewpointC'].head()

0    3
1    3
2    4
3    4
4    5
Name: Min_DewpointC, dtype: int64

Now for every column in both `joined` and `joined_test`, check to see if it has the `_y` suffix, and if so, drop it. Warning: a data frame can have duplicate column names, but calling `df.drop` will drop _all_ instances with the passed-in column name! This could lead to calling drop a second time on a column that no longer exists!

In [58]:
for i in (joined, joined_test):
    for j in i.columns:
        if j in i.columns and ('_y' in j or '_g' in j or '_rf' in j): i.drop(j, axis=1, inplace=True)

In [59]:
joined.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'State', 'file', 'week', 'trend',
       'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
       'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC', 'Max_Humidity',
       'Mean_Humidity', 'Min_Humidity', 'Max_Sea_Level_PressurehPa',
       'Mean_Sea_Level_PressurehPa', 'Min_Sea_Level_PressurehPa',
       'Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'Max_Gust_SpeedKm_h',
       'Precipitationmm', 'CloudCover', 'Events', 'WindDirDegrees',
       'StateName'],
      d

In [60]:
joined_test.columns

Index(['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
       'week', 'trend', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Sea_Level_PressurehPa', 'Max_VisibilityKm', 'Mean_VisibilityKm',
       'Min_VisibilitykM', 'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h',
       'Max_Gust_SpeedKm_h', 'Precipitationmm', 'CloudCover', 'Events',
       'WindDirDegrees', 'StateName'],
      dtype='object')

In [61]:
len(joined.columnsumns)

55

In [62]:
len(joined_test.columns)

54

For the columns `CompetitionOpenSinceYear`, `CompetitionOpenSinceMonth`, `Promo2SinceYear`, and `Promo2SinceMonth`, replace `NA` values with the following values (respectively):
- 1900
- 1
- 1900
- 1

In [64]:
for i in (joined, joined_test):
    i.loc[i['CompetitionOpenSinceYear'].isna(), 'CompetitionOpenSinceYear'] = 1900
    i.loc[i['CompetitionOpenSinceMonth'].isna(), 'CompetitionOpenSinceMonth'] = 1
    i.loc[i['Promo2SinceYear'].isna(), 'Promo2SinceYear'] = 1900
    i.loc[i['Promo2SinceWeek'].isna(), 'Promo2SinceWeek'] = 1

In [68]:
joined.head()['Promo2SinceYear']

0    1900.0
1    2010.0
2    2011.0
3    1900.0
4    1900.0
Name: Promo2SinceYear, dtype: float64

Create a new field `CompetitionOpenSince` that converts `CompetitionOpenSinceYear` and `CompetitionOpenSinceMonth` and maps them to a specific date. Then create a new field `CompetitionDaysOpen` that subtracts `CompetitionOpenSince` from `Date`. 

In [73]:
pd.to_datetime({'year': [2019], 'month': [1], 'day': [1]})

0   2019-01-01
dtype: datetime64[ns]

In [78]:
joined['Date'].dtype

dtype('O')

In [92]:
import datetime
for i in (joined, joined_test):
    i['CompetitionOpenSince'] = pd.to_datetime({'year': i['CompetitionOpenSinceYear'], 
                                                'month': i['CompetitionOpenSinceMonth'],
                                                'day': 15})
    i['CompetitionDaysOpen'] = (pd.to_datetime(i['Date']) - i['CompetitionOpenSince']) / datetime.timedelta(days=1)
    

In [93]:
joined[['CompetitionOpenSince', 'CompetitionDaysOpen']].head()

,CompetitionOpenSince,CompetitionDaysOpen
0,2008-09-15,2510.0
1,2007-11-15,2815.0
2,2006-12-15,3150.0
3,2009-09-15,2145.0
4,2015-04-15,107.0


For `CompetitionDaysOpen`, replace values where `CompetitionDaysOpen < 0` with 0, and cases where `CompetitionOpenSinceYear < 1990` with 0.

In [94]:
joined['CompetitionOpenSinceYear'].dtype, joined['CompetitionDaysOpen'].dtype

(dtype('float64'), dtype('float64'))

In [97]:
for i in (joined, joined_test):
    i['CompetitionDaysOpen'][i['CompetitionDaysOpen'] < 0] = 0
    i['CompetitionOpenSinceYear'][i['CompetitionOpenSinceYear'] < 1990] = 1990

/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [98]:
joined[['CompetitionDaysOpen', 'CompetitionOpenSinceYear']]

,CompetitionDaysOpen,CompetitionOpenSinceYear
0,2510.0,2008.0
1,2815.0,2007.0
2,3150.0,2006.0
3,2145.0,2009.0
4,107.0,2015.0
5,593.0,2013.0
6,837.0,2013.0
7,289.0,2014.0
8,5463.0,2000.0
9,2145.0,2009.0


We add "CompetitionMonthsOpen" field, limiting the maximum to 2 years to limit number of unique categories.

In [107]:
for i in (joined, joined_test):
    i['CompetitionMonthsOpen'] = i['CompetitionDaysOpen'] // 30
    i['CompetitionMonthsOpen'] = i.apply(lambda x: 24 if x['CompetitionMonthsOpen'] > 24 else x['CompetitionMonthsOpen'], axis=1)

In [112]:
joined['CompetitionMonthsOpen'].value_counts()

24.0    790886
0.0      92590
6.0       6356
7.0       6350
4.0       6278
5.0       6261
8.0       6203
9.0       6138
10.0      6118
11.0      5971
12.0      5867
3.0       5850
13.0      5724
15.0      5699
14.0      5655
19.0      5649
2.0       5621
18.0      5619
16.0      5618
22.0      5558
17.0      5549
20.0      5500
1.0       5482
21.0      5398
23.0      5269
Name: CompetitionMonthsOpen, dtype: int64

Same process for Promo dates. You may need to install the `isoweek` package first.

In [28]:
# If needed, uncomment:
# ! pip install isoweek

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Use the `isoweek` package to turn `Promo2Since` to a specific date -- the Monday of the week specified in the column. Compute a field `Promo2SinceDays` that subtracts the current date from the `Promo2Since` date.

In [116]:
from isoweek import Week
for i in (joined, joined_test):
    i['Promo2Since'] = i.apply(lambda x: Week(int(x['Promo2SinceYear']), int(x['Promo2SinceWeek'])).monday(), axis=1)

In [117]:
joined['Promo2Since'].head(25)

0     1900-01-01
1     2010-03-29
2     2011-04-04
3     1900-01-01
4     1900-01-01
5     1900-01-01
6     1900-01-01
7     1900-01-01
8     1900-01-01
9     1900-01-01
10    2012-01-02
11    2010-03-29
12    2009-11-02
13    2011-10-03
14    2011-04-04
15    1900-01-01
16    2010-06-28
17    2012-04-02
18    2011-05-30
19    2014-09-29
20    2009-11-02
21    2012-05-28
22    1900-01-01
23    2011-10-03
24    1900-01-01
Name: Promo2Since, dtype: object

Perform the following modifications on both the train and test set:
- For cases where `Promo2Days` is negative or `Promo2SinceYear` is before 1990, set `Promo2Days` to 0
- Create `Promo2Weeks
- For cases where `Promo2Weeks` is negative, set `Promo2Weeks` to 0
- For cases where `Promo2Weeks` is above 25, set `Promo2Weeks` to 25

Print the number of unique values for `Promo2Weeks` in training and test df's.

Pickle `joined` to `PATH/'joined'` and `joined_test` to `PATH/'joined_test'`.

## Durations

Write a function `get_elapsed` that takes arguments `fld` (a boolean field) and `pre` (a prefix to be appended to `fld` in a new column representing the days until/since the event in `fld`), and adds a column `pre+fld` representing the date-diff (in days) between the current date and the last date `fld` was true.

We'll be applying this to a subset of columns:

Create a variable `columns` containing the strings: 
- Date
- Store
- Promo
- StateHoliday
- SchoolHoliday

These will be the fields on which we'll be computing elapsed days since/until.

Create one big dataframe with both the train and test sets called `df`.

Sort by `Store` and `Date` ascending, and use `add_elapsed` to get the days since the last `SchoolHoliday` on each daya. Reorder by `Store` ascending and `Date` descending to get the days _until_ the next `SchoolHoliday`.

Do the same for `StateHoliday`.

Do the same for `Promo`.

Set the index on `df` to `Date`.

Reassign `columns` to `['SchoolHoliday', 'StateHoliday', 'Promo']`.

For columns `Before/AfterSchoolHoliday`, `Before/AfterStateHoliday`, and `Before/AfterPromo`, fill null values with 0.

Create a dataframe `bwd` that gets 7-day backward-rolling sums of the columns in `columns`, grouped by `Store`.

Create a dataframe `fwd` that gets 7-day forward-rolling sums of the columns in `columns`, grouped by `Store`.

Show the head of `bwd`.

Show the head of `fwd`.

Drop the `Store` column from `fwd` and `bwd` inplace, and reset the index inplace on each.

Reset the index on `df`.

Merge `df` with `bwd` and `fwd`.

Drop `columns` from df inplace -- we don't need them anymore, since we've captured their information in columns with types more suitable for machine learning.

Print out the head of `df`.

Pickle `df` to `PATH/'df'`.

Cast the `Date` column to a datetime column.

Join `joined` with `df` on `['Store', 'Date']`.

This is not necessarily the best idea, but the authors removed all examples for which sales were equal to zero. If you're trying to stay true to what the authors did, do that now.

Reset the indices, and pickle train and test to `train_clean` and `test_clean`.